In [3]:
#include "../json/include/nlohmann/json.hpp"
using json = nlohmann::json;

std::vector<std::string> strategies = {
  "geant4_100muon_100GeV",
  "geant4_100muon_10GeV"
};

// auto cRate = new TCanvas("cRate", "Rate vs Threads and Slots", 2400, 3200);
// cRate->Divide(3, 4);

// std::ifstream resultFile("schedule_simple_H100_no_numa_longrun2_straightLaunches.json");
// json resultJson = json::parse(resultFile);
// resultFile.close();

// std::cout << "Parsed JSON with " << resultJson["results"].size() << " results." << std::endl;

struct MeasurementPoint {
  int threads;
  int streams;
  bool operator<(const MeasurementPoint &other) const {
    return std::tie(threads, streams) < std::tie(other.threads, other.streams);
  }
};

TH1::StatOverflows(kTRUE) ;
// Map[strategy][MeasurementPoint] -> TH1D
std::map<std::string, std::map<MeasurementPoint, TH1D>> throughputMap;

// Fill the throughputMap with data from JSON files
for (const auto& strategy : strategies) {
    std::string filename = "traccc_short_" + strategy + ".json";
    std::ifstream resultFile(filename);
    if (!resultFile.is_open()) {
      std::cerr << "Could not open file: " << filename << " - skipping." << std::endl;
      continue;
    }
    json resultJson;
    try {
      resultJson = json::parse(resultFile);
      resultFile.close();
    } catch (const std::exception& e) {
      std::cerr << "Error parsing JSON from " << filename << ": " << e.what() << " - skipping." << std::endl;
      resultFile.close();
      continue;
    }
    
    if (!resultJson.contains("results")) {
      std::cerr << "No 'results' field in " << filename << " - skipping." << std::endl;
      continue;
    }
    
    std::cout << "Parsed JSON with " << resultJson["results"].size() << " results from " << filename << "." << std::endl;
    for (auto r: resultJson["results"]) {
      int t = std::stoi(r["threads"].dump());
      int s = std::stoi(r["streams"].dump());
      double thr = std::stod(r["throughput"].dump());

      // std::cout << "Threads: " << t << " Streams: " << s << " Throughput: " << thr << std::endl;
      auto &h = throughputMap[strategy][{t, s}];
      h.SetName(("Threads_" + std::to_string(t) + "_Streams_" + std::to_string(s) + "_" + strategy).c_str());
      //h.ResetStats();
      h.Fill(thr, 1);
    }
}

// Compute min and max rates across all strategies and NUMA configurations
double minRate = std::numeric_limits<double>::max();
double maxRate = std::numeric_limits<double>::lowest();
int minThreads = std::numeric_limits<int>::max();
int maxThreads = std::numeric_limits<int>::lowest();
int minStreams = std::numeric_limits<int>::max();
int maxStreams = std::numeric_limits<int>::lowest();
for (const auto& [strategy, histMap] : throughputMap) {
  for (const auto& [key, hist] : histMap) {
    minRate = std::min(minRate, hist.GetMean());
    maxRate = std::max(maxRate, hist.GetMean());
    minThreads = std::min(minThreads, key.threads);
    maxThreads = std::max(maxThreads, key.threads);
    minStreams = std::min(minStreams, key.streams);
    maxStreams = std::max(maxStreams, key.streams);
  }
}

// Create graphs for each strategy and NUMA configuration
// minMax has the complete set of keys
  size_t pass=0, passLast = strategies.size();
  gStyle->SetPalette(1);
for (const auto& strategy : strategies) {
  // Skip strategies with no data
  if (throughputMap.find(strategy) == throughputMap.end() || throughputMap[strategy].empty()) {
    std::cerr << "No data for strategy: " << strategy << " - skipping plot." << std::endl;
    continue;
  }
  
  auto cRate = new TCanvas(("cRate_" + strategy).c_str(), ("Rate vs Threads and Slots (" + strategy + ")").c_str(), 1600  , 800);
  cRate->Divide(3,1);
  auto gflat = new TGraph2D();
  auto g3d = new TGraph2D();
  auto g3dErr = new TGraph2DErrors();
  gflat->SetTitle(("Rate vs Threads and Slots (" + strategy + ");Slots;Threads;Rate (evts/s)").c_str());
  g3d->SetTitle(("Rate vs Threads and Slots (" + strategy + ");Slots;Threads;Rate (evts/s)").c_str());
  g3dErr->SetTitle(("Rate vs Threads and Slots (" + strategy + ");Slots;Threads;Rate (evts/s)").c_str());
  gflat->GetHistogram()->SetBins(100, minStreams, maxStreams, 100, minThreads, maxThreads);
  g3d->GetHistogram()->SetBins(100, minStreams, maxStreams, 100, minThreads, maxThreads);
  g3dErr->GetHistogram()->SetBins(100, minStreams, maxStreams, 100, minThreads, maxThreads);
  int i = 0;
  for (const auto& [key, hist] : throughputMap[strategy]) {
    if (key.streams == 0 || key.threads == 0) continue; // Skip invalid points
    // std::cout << "Strategy: " << strategy << " NUMA: " << numa << " Threads: " << key.threads << " Streams: " << key.streams 
    //           << " Mean Throughput: " << hist.GetMean() 
    //           << " StdDev Throughput: " << hist.GetStdDev() 
    //           << " Entries: " << hist.GetEntries() << std::endl;
    auto s = key.streams;
    auto t = key.threads;
    gflat->SetPoint(i, s, t, hist.GetMean());
    g3d->SetPoint(i, s, t, hist.GetMean());
    g3dErr->SetPoint(i, s, t, hist.GetMean());
    g3dErr->SetPointError(i, 0, 0, hist.GetStdDev());
    i++;
  }

  cRate->cd(1);
  gflat->Draw("COLZ");
  cRate->cd(2);
  g3d->Draw("SURF1");
  cRate->cd(3);
  g3dErr->Draw("P0COL ERR");
  cRate->Update();
  // if (!pass++)  cRate->Print("evt_rateexport.pdf(");
  // else if (pass < passLast) cRate->Print("evt_rateexport.pdf");
  // else { cRate->Print("evt_rateexport.pdf)"); cRate->Close(); break; }
  cRate->Print(("evt_short_rateexport_" + strategy + ".png").c_str());
  cRate->Close();
}


Parsed JSON with 40 results from traccc_short_geant4_100muon_100GeV.json.
Parsed JSON with 40 results from traccc_short_geant4_100muon_10GeV.json.


Info in <TCanvas::Print>: png file evt_short_rateexport_geant4_100muon_100GeV.png has been created
Info in <TCanvas::Print>: png file evt_short_rateexport_geant4_100muon_10GeV.png has been created


In [4]:
#include "../json/include/nlohmann/json.hpp"
using json = nlohmann::json;
  
std::vector<std::string> all_strategies = {
  "geant4_100muon_100GeV",
  "geant4_100muon_10GeV"
};

std::vector<std::string> plot_strategies = {
  "geant4_100muon_100GeV",
  "geant4_100muon_10GeV"
};

struct MeasurementPoint {
  int threads;
  int streams;
  bool operator<(const MeasurementPoint &other) const {
    return std::tie(threads, streams) < std::tie(other.threads, other.streams);
  }
};

TH1::StatOverflows(kTRUE) ;
// Map[strategy][MeasurementPoint] -> TH1D
std::map<std::string, std::map<MeasurementPoint, TH1D>> throughputMap;

// Fill the throughputMap with data from JSON files
for (const auto& strategy : all_strategies) {
    std::string filename = "traccc_short_" + strategy + ".json";
    std::ifstream resultFile(filename);
    if (!resultFile.is_open()) {
      std::cerr << "Could not open file: " << filename << " - skipping." << std::endl;
      continue;
    }
    json resultJson;
    try {
      resultJson = json::parse(resultFile);
      resultFile.close();
    } catch (const std::exception& e) {
      std::cerr << "Error parsing JSON from " << filename << ": " << e.what() << " - skipping." << std::endl;
      resultFile.close();
      continue;
    }
    
    if (!resultJson.contains("results")) {
      std::cerr << "No 'results' field in " << filename << " - skipping." << std::endl;
      continue;
    }
    
    std::cout << "Parsed JSON with " << resultJson["results"].size() << " results from " << filename << "." << std::endl;
    for (auto r: resultJson["results"]) {
      int t = std::stoi(r["threads"].dump());
      int s = std::stoi(r["streams"].dump());
      double thr = std::stod(r["throughput"].dump());

      // std::cout << "Threads: " << t << " Streams: " << s << " Throughput: " << thr << std::endl;
      auto &h = throughputMap[strategy][{t, s}];
      h.SetName(("Threads_" + std::to_string(t) + "_Streams_" + std::to_string(s) + "_" + strategy).c_str());
      //h.ResetStats();
      h.Fill(thr, 1);
    }
}

// Filter plot_strategies to only include those with data
std::vector<std::string> valid_plot_strategies;
for (const auto& strategy : plot_strategies) {
  if (throughputMap.find(strategy) != throughputMap.end() && !throughputMap[strategy].empty()) {
    valid_plot_strategies.push_back(strategy);
  } else {
    std::cerr << "No data for strategy: " << strategy << " - excluding from plots." << std::endl;
  }
}

if (valid_plot_strategies.empty()) {
  std::cerr << "No valid strategies with data found. Cannot create plots." << std::endl;
} else {
  std::cout << "Creating plots for " << valid_plot_strategies.size() << " strategies with data." << std::endl;

  // Compute min and max rates across all strategies and NUMA configurations
  double minRate = 0.0l; //std::numeric_limits<double>::max();
  double maxRate = std::numeric_limits<double>::lowest();
  int minThreads = std::numeric_limits<int>::max();
  int maxThreads = std::numeric_limits<int>::lowest();
  int minStreams = std::numeric_limits<int>::max();
  int maxStreams = std::numeric_limits<int>::lowest();
  for (const auto& [strategy, histMap] : throughputMap) {
    for (const auto& [key, hist] : histMap) {
      minRate = std::min(minRate, hist.GetMean());
      maxRate = std::max(maxRate, hist.GetMean());
      minThreads = std::min(minThreads, key.threads);
      maxThreads = std::max(maxThreads, key.threads);
      minStreams = std::min(minStreams, key.streams);
      maxStreams = std::max(maxStreams, key.streams);
    }
  }

  // Create canvas with 2 columns, 3 plots per strategy
  int plotSize = 450; // Size of each plot in pixels
  int nCols = 6; // 2 columns × 3 plots per strategy
  int nRows = (valid_plot_strategies.size() + 1) / 2; // Ceiling division for 2 strategies per row
  auto cRate = new TCanvas("cRate", "Rate vs Threads and Slots", nCols*plotSize, nRows * plotSize);
  cRate->Divide(nCols, nRows);

  // Create graphs for each strategy
  for (size_t idx = 0; idx < valid_plot_strategies.size(); idx++) {
    const auto& strategy = valid_plot_strategies[idx];
    
    auto gflat = new TGraph2D();
    auto g3d = new TGraph2D();
    auto g3dErr = new TGraph2DErrors();
    gflat->SetTitle(("Rate vs Threads and Slots (" + strategy + ");Slots;Threads;Rate (evts/s)").c_str());
    g3d->SetTitle(("Rate vs Threads and Slots (" + strategy + ");Slots;Threads;Rate (evts/s)").c_str());
    g3dErr->SetTitle(("Rate vs Threads and Slots (" + strategy + ");Slots;Threads;Rate (evts/s)").c_str());
    gflat->GetHistogram()->SetBins(100, minStreams, maxStreams, 100, minThreads, maxThreads);
    g3d->GetHistogram()->SetBins(100, minStreams, maxStreams, 100, minThreads, maxThreads);
    g3dErr->GetHistogram()->SetBins(100, minStreams, maxStreams, 100, minThreads, maxThreads);
    
    int i = 0;
    double best=0;
    MeasurementPoint bestKey = {0, 0};
    for (const auto& [key, hist] : throughputMap[strategy]) {
      if (key.streams == 0 || key.threads == 0) continue; // Skip invalid points
      auto s = key.streams;
      auto t = key.threads;
      gflat->SetPoint(i, s, t, hist.GetMean());
      g3d->SetPoint(i, s, t, hist.GetMean());
      g3dErr->SetPoint(i, s, t, hist.GetMean());
      g3dErr->SetPointError(i, 0, 0, hist.GetStdDev());
      if (hist.GetMean() > best) {
        best = hist.GetMean();
        bestKey = key;
      }
      i++;
    }
    std::cout << "Strategy: " << strategy << " Best Threads: " << bestKey.threads 
              << " Streams: " << bestKey.streams 
              << " Mean Throughput: " << best 
              << " StdDev Throughput: " << throughputMap[strategy][bestKey].GetStdDev() << std::endl;
    
    gflat->SetMinimum(minRate * 0.9);
    gflat->SetMaximum(maxRate * 1.1);
    g3d->SetMinimum(minRate * 0.9);
    g3d->SetMaximum(maxRate * 1.1);
    g3dErr->SetMinimum(minRate * 0.9);
    g3dErr->SetMaximum(maxRate * 1.1);
    
    // Layout: 2 strategies per row, 3 plots each
    // Row layout: [colz1] [surf1] [err1] [colz2] [surf2] [err2]
    int row = idx / 2;
    int col = (idx % 2) * 3; // 0 for left strategy, 3 for right strategy
    
    cRate->cd(nCols * row + col + 1);
    gflat->Draw("COLZ");
    cRate->cd(nCols * row + col + 2);
    g3d->Draw("SURF1");
    cRate->cd(nCols * row + col + 3);
    g3dErr->Draw("P0COL ERR");
  }

  gStyle->SetPalette(1);
  cRate->Draw();
}


Parsed JSON with 40 results from traccc_short_geant4_100muon_100GeV.json.
Parsed JSON with 40 results from traccc_short_geant4_100muon_10GeV.json.
Creating plots for 2 strategies with data.
Strategy: geant4_100muon_100GeV Best Threads: 4 Streams: 6 Mean Throughput: 383.112 StdDev Throughput: 3.34924
Strategy: geant4_100muon_10GeV Best Threads: 4 Streams: 6 Mean Throughput: 359.309 StdDev Throughput: 2.64755
